In [4]:
!pip install snowflake-snowpark-python

                                              0.0/331.0 kB ? eta -:--:--
     ------------                           112.6/331.0 kB 3.3 MB/s eta 0:00:01
     -------------------------------------  327.7/331.0 kB 3.4 MB/s eta 0:00:01
     -------------------------------------- 331.0/331.0 kB 2.9 MB/s eta 0:00:00
                                              0.0/877.1 kB ? eta -:--:--
     ------                                 153.6/877.1 kB 3.1 MB/s eta 0:00:01
     ------------------                     430.1/877.1 kB 4.5 MB/s eta 0:00:01
     ---------------------------            624.6/877.1 kB 4.4 MB/s eta 0:00:01
     -------------------------------------- 877.1/877.1 kB 5.0 MB/s eta 0:00:00
                                              0.0/105.0 kB ? eta -:--:--
     -------------------------------------- 105.0/105.0 kB 5.9 MB/s eta 0:00:00
  Using cached platformdirs-3.11.0-py3-none-any.whl (17 kB)
  Attempting uninstall: platformdirs
    Found existing installation: platformdir

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
black 0.0 requires click>=8.0.0, but you have click 7.1.2 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.


In [41]:
%%writefile testmapkoreng.py

import json
import streamlit as st
import pandas as pd
import folium
from streamlit_folium import folium_static
import requests
from datetime import date, datetime
import plotly.express as px
import numpy as np
from typing import Iterable
from snowflake.snowpark.session import Session
import altair as alt
from altair import datum
import folium
from altair import Chart

# 데이터 로딩
data = pd.read_excel("stream_data.xlsx")

class OurFilter:
    def __init__(self, human_name, table_column, widget_id, widget_type):
        self.human_name = human_name
        self.table_column = table_column
        self.widget_id = widget_id
        self.widget_type = widget_type
        self.enabled = False  # Initialize enabled attribute
        self.value = None  # Initialize value attribute

    def enable(self):
        self.enabled = True

    def create_widget(self):
        if self.widget_type == st.checkbox:
            self.value = st.checkbox(self.human_name, key=self.widget_id)
        elif self.widget_type == st.date_input:
            self.value = st.date_input(self.human_name, key=self.widget_id)
        elif self.widget_type == st.multiselect:
            options = sorted(map(str, data[self.table_column].unique()))
            # Remove the default value from here
            self.value = st.multiselect(self.human_name, options, key=self.widget_id)
    

# Define a custom SessionState class
class SessionState:
    def __init__(self, **kwargs):
        for key, value in kwargs.items():
            setattr(self, key, value)

# Initialize session state with default values
if 'start_date_widget' not in st.session_state:
    st.session_state.start_date_widget = data['date'].min()

if 'end_date_widget' not in st.session_state:
    st.session_state.end_date_widget = data['date'].max()

if 'topics_widget' not in st.session_state:
    st.session_state.topics_widget = data['topics'].unique().tolist()

def init_connection():
    # Placeholder for initializing connection
    return None

def _get_human_filter_names(filters):
    return [f.human_name for f in filters]

def _is_any_filter_enabled():
    return any(f.enabled for f in st.session_state.filters)

def _get_active_filters():
    return [f for f in st.session_state.filters if f.enabled]

def draw_sidebar():
    """Should include dynamically generated filters"""

    with st.sidebar:
        selected_filters = st.multiselect(
            "Select which filters to enable",
            list(_get_human_filter_names(st.session_state.filters)),
            [],
        )
        for _f in st.session_state.filters:
            if _f.human_name in selected_filters:
                _f.enable()

        if _is_any_filter_enabled():
            with st.form(key="input_form"):

                for _f in _get_active_filters():
                    _f.create_widget()
                st.session_state.clicked = st.form_submit_button(label="Submit")
        else:
            st.write("Please enable a filter")

if __name__ == "__main__":
    # Initialize the filters
    session = init_connection()
    OurFilter.session = session
    MY_TABLE = "news_data"
    OurFilter.table_name = MY_TABLE

    # Dynamically generate filters based on selected columns
    st.session_state.filters = [
        OurFilter(
            human_name="Start date",
            table_column="date",
            widget_id="start_date_widget",
            widget_type=st.date_input,
        ),
        OurFilter(
            human_name="End date",
            table_column="date",
            widget_id="end_date_widget",
            widget_type=st.date_input,
        ),
        OurFilter(
            human_name="Choose topics",
            table_column="topics",
            widget_id="topics_widget",
            widget_type=st.multiselect,
        )
    ]
    # # Explicitly initialize session state values
    # st.session_state.start_date_widget = data['일자'].min()
    # st.session_state.end_date_widget = data['일자'].max()
    # st.session_state.topics_widget = ['전쟁']
    
    draw_sidebar()
    
     # Access the selected values
    start_date_widget = st.session_state.start_date_widget
    end_date_widget = st.session_state.end_date_widget
    topics_widget = st.session_state.topics_widget
    
    # 한국 도시와 외국 도시 데이터를 나누어 저장
    korea_json = "한국시도별.json"
    korea_json = json.load(open(korea_json))

    korea_center = pd.read_excel("한국시도별중심마커.xlsx")
    nation_center = pd.read_excel("국가별중심마커.xlsx")
    
    # tabs 만들기
    tab1, tab2, tab3 = st.tabs(["국내", "해외", "기부"])

    # tab1 내용물 구성하기
    with tab1:
        # 제목 넣기
        st.markdown("## 국내 재난/재해 상황")

        def draw_map(start, end, topic) :
            
            start_date = np.datetime64(start)
            end_date = np.datetime64(end)

            df = data.copy()
            return_df = df[data["topics"].isin(topic)]
            return_df = return_df.loc[(return_df.date >= start_date) & (return_df.date <= end_date), :] ### choropleth map을 그리기 위한 데이터 프레임
            return_df = return_df.groupby("city", 
                                          as_index = False)[["risk"]].\
                                                sum().sort_values("risk", ascending = False).reset_index(drop = True)

            ## 지도 시작 중심값을 알리기 위해 가장 위험도가 높은 도시의 중심 좌표 구하기
            high_risk_nation = return_df["city"][0]

            marker_info = pd.merge(return_df, korea_center, how = "right", on = "city")
            # 값의 nan값이 존재하면 안돼요
            marker_info = marker_info.dropna(subset=["위도", "경도"])

            m1 = folium.Map([36, 128], tiles = "cartodbpositron", 
                            zoom_start = 7, scrollWhellZoom = False)

            choro1 = folium.Choropleth(
                            data = return_df,
                            geo_data = korea_json,
                            columns = ["city","risk"],
                            key_on = "feature.properties.name",
                            mapbox_style = "carto_positron",
                            fill_color = "YlOrRd"
                        ).add_to(m1)
            folium_static(m1)

            for i in range(len(marker_info)) : 
                city = marker_info["city"][i]
                line_data = data.loc[data["city"] == city,]
                line = (
                Chart(line_data, title = f"risk trend in {city}")
                .mark_line(point = True, opacity = 0.7).\
                    encode(x = "date", y = "risk",)
                    )
                vega_lite = folium.VegaLite(
                    line, width = "70%", height = "70%"
                )

                marker = folium.Marker(
                    location = [marker_info["위도"][i], marker_info["경도"][i]],
                    icon = folium.Icon(color = "red")
                )
                popup = folium.Popup()

                vega_lite.add_to(popup)
                popup.add_to(marker)
                marker.add_to(m1)
            return m1
            
        # 사용자 입력을 기반으로 지도 그리기
        draw_map(st.session_state.start_date_widget, st.session_state.end_date_widget, st.session_state.topics_widget)

        col1, col2 = st.columns([5, 5])
        korean_data = data[data['외국유무'] == '한국']
        with col1:
            topics_distribution = korean_data['topics'].value_counts().reset_index()
            fig_topics = px.pie(
                topics_distribution,
                names='index',
                values='topics',
                title='한국 Topics 분포',
                hole=0.4,  # 도넛 차트 설정
                color_discrete_sequence=px.colors.qualitative.Set3  # 색상 조정
            )
            fig_topics.update_layout(
                width=400,
                height=400,
                title_x=0.5,
                margin=dict(l=0, r=0, t=30, b=0),
                showlegend=True,  # 범례 표시
                legend_title="Topics" 
            )
            st.plotly_chart(fig_topics)

    # 가운데 열에는 한국 도시별 리스크 그래프 표시
        with col2:
            korean_top_5 = korean_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
            fig_korean = px.bar(
                korean_top_5,
                y='risk',
                x='city',
                orientation='v',
                title="Top 5 한국 도시별 리스크",
                color='risk',  # 색상을 리스크에 따라 설정
                color_continuous_scale='viridis'  # 색상 맵 설정
            )
            fig_korean.update_layout(
                width=400,
                height=400,
                title_x=0.5,
                margin=dict(l=0, r=0, t=30, b=0),
                showlegend=True
            )
            st.plotly_chart(fig_korean)


         # 상위 국가 선택을 위한 라디오 버튼

        selected_country = st.selectbox("Select a country", korean_top_5['city'].head(5))
        # st.write(f"Selected country: {selected_country}")

        # 선택한 국가에 해당하는 데이터프레임을 리스크 큰 순으로 정렬하여 상위 5개 행만 선택
        df_selected_country_sorted = korean_data[korean_data['city'] == selected_country][['doc_id', 'city', 'date', '제목', '요약', 'risk']].sort_values('risk', ascending=False).head(5)
        col1, col2 = st.columns([2,8])
        with col1:
        # doc_id 라디오 버튼 생성
            selected_doc_id = st.radio("Select a news", df_selected_country_sorted['doc_id'].tolist())
        with col2:
            # 선택된 doc_id에 해당하는 HTML 요약을 표시하는 함수
            def show_article_html(doc_id):
                selected_title = korean_data[korean_data['doc_id'] == doc_id]['제목'].iloc[0]
                selected_article = korean_data[korean_data['doc_id'] == doc_id]['요약'].iloc[0]

                # HTML 스타일 및 타이틀 수정
                modified_html = f'''
                <html>
                    <head>
                        <title>{selected_title}</title>
                        <style>
                            /* 원하는 스타일을 여기에 추가하세요 */
                            body {{
                                font-family: "Arial", sans-serif;
                                font-size: 16px;
                                line-height: 1.6;
                                color: #333;
                            }}
                            .title {{
                                font-size: 24px;
                                font-weight: bold;
                            }}
                            .text {{
                                font-size: 18px;
                                font-style: italic;
                            }}
                        </style>
                    </head>
                    <body>
                        <h1 class="title">{selected_title}</h1>
                        <div class="text">{selected_article}</div>
                    </body>
                </html>
                '''

                st.markdown(modified_html, unsafe_allow_html=True)

            # 라디오 버튼을 클릭하면 해당 doc_id에 대한 HTML을 표시
            show_article_html(selected_doc_id)  
    

    # tab2 내용물 구성하기
    with tab2:
        # 제목 넣기
        st.markdown("## 해외 재난/재해 상황")
       
        nation_json = "nation.json"
        nation_json = json.load(open(nation_json))

        def draw_map_eng(start, end, topic) :
            
            # draw_map 함수 정의
            
            start_date = np.datetime64(start)
            end_date = np.datetime64(end)
            
            df = data.copy()
            return_df = df[data["topics"].isin(topic)]
            return_df = return_df.loc[(return_df.date >= start_date) & (return_df.date <= end_date), :] ### choropleth map을 그리기 위한 데이터 프레임
            return_df = return_df.groupby("city", 
                                          as_index = False)[["risk"]].\
                                                sum().sort_values("risk", ascending = False).reset_index(drop = True)

            ## 지도 시작 중심값을 알리기 위해 가장 위험도가 높은 도시의 중심 좌표 구하기
            high_risk_nation = return_df["city"][0]

            map_center = nation_center.loc[nation_center["city"] == high_risk_nation, ["위도", "경도"]].reset_index(drop=True)
            center_long, center_lati = map_center["위도"][0], map_center["경도"][0]

            marker_info = pd.merge(return_df, nation_center, how = "right", on = "city")

            m2 = folium.Map([center_long, center_lati], tiles = "cartodbpositron", 
                            zoom_start = 5.5, scrollWhellZoom = False)

            choro2 = folium.Choropleth(
                            data = return_df,
                            geo_data = nation_json,
                            columns = ["city","risk"],
                            key_on = "feature.properties.name",
                            mapbox_style = "carto_positron",
                            fill_color = "YlOrRd"
                        ).add_to(m2)
            folium_static(m2)

            for i in range(len(marker_info)) : 
                city = marker_info["city"][i]
                line_data = data.loc[data["city"] == city,]
                line = (
                Chart(line_data, title = f"risk trend in {city}")
                .mark_line(point = True, opacity = 0.7).\
                    encode(x = "date", y = "risk",)
                    )
                vega_lite = folium.VegaLite(
                    line, width = "70%", height = "70%"
                )

                marker = folium.Marker(
                    location = [marker_info["위도"][i], marker_info["경도"][i]],
                    icon = folium.Icon(color = "red")
                )
                popup = folium.Popup()

                vega_lite.add_to(popup)
                popup.add_to(marker)
                marker.add_to(m2)

            return m2

        draw_map_eng(st.session_state.start_date_widget, st.session_state.end_date_widget, st.session_state.topics_widget)
        
     # 컬럼 설정
        col1, col2 = st.columns([5, 5])
        foreign_data = data[data['외국유무'] == '외국']
        with col1:
            topics_distribution = foreign_data['topics'].value_counts().reset_index()
            fig_topics = px.pie(
                topics_distribution,
                names='index',
                values='topics',
                title='외국 Topics 분포',
                hole=0.4,  # 도넛 차트 설정
                color_discrete_sequence=px.colors.qualitative.Set3  # 색상 조정
            )
            fig_topics.update_layout(
                width=400,
                height=400,
                title_x=0.5,
                margin=dict(l=0, r=0, t=30, b=0),
                showlegend=True,  # 범례 표시
                legend_title="Topics" 
            )
            st.plotly_chart(fig_topics)

        with col2:
            foreign_top_5 = foreign_data.groupby('city')['risk'].sum().nlargest(5).reset_index()
            fig_foreign = px.bar(
                foreign_top_5,
                y='risk',
                x='city',
                orientation='v',
                title="Top 5 외국 도시별 리스크",
                color='risk',
                color_continuous_scale='viridis'
            )
            fig_foreign.update_layout(
                width=400,
                height=400,
                title_x=0.5,
                margin=dict(l=0, r=0, t=30, b=0),
                showlegend=True
            )
            st.plotly_chart(fig_foreign)

            ##########################################뭐가 문제냐
        selected_country_eng = st.selectbox("Select a country", foreign_top_5['city'].head(5))
        # st.write(f"Selected country: {selected_country_eng}")

            # 선택한 국가에 해당하는 데이터프레임을 리스크 큰 순으로 정렬하여 상위 5개 행만 선택
        df_selected_country_sorted_eng = foreign_data[foreign_data['city'] == selected_country_eng][['doc_id', 'city', 'date', '제목', '요약', 'risk']].sort_values('risk', ascending=False).head(5)
        col1, col2 = st.columns([2,8])
        with col1:
            # doc_id 라디오 버튼 생성
            selected_doc_id_eng = st.radio("Select a news", df_selected_country_sorted_eng['doc_id'].tolist())
        with col2:
                # 선택된 doc_id에 해당하는 HTML 요약을 표시하는 함수
            def show_article_html(doc_id):
                    selected_title_eng = foreign_data[foreign_data['doc_id'] == doc_id]['제목'].iloc[0]
                    selected_article_eng = foreign_data[foreign_data['doc_id'] == doc_id]['요약'].iloc[0]

                    # HTML 스타일 및 타이틀 수정
                    modified_html_eng = f'''
                    <html>
                        <head>
                            <title>{selected_title_eng}</title>
                            <style>
                                /* 원하는 스타일을 여기에 추가하세요 */
                                body {{
                                    font-family: "Arial", sans-serif;
                                    font-size: 16px;
                                    line-height: 1.6;
                                    color: #333;
                                }}
                                .title {{
                                    font-size: 24px;
                                    font-weight: bold;
                                }}
                                .text {{
                                    font-size: 18px;
                                    font-style: italic;
                                }}
                            </style>
                        </head>
                        <body>
                            <h1 class="title">{selected_title_eng}</h1>
                            <div class="text">{selected_article_eng}</div>
                        </body>
                    </html>
                    '''

                    st.markdown(modified_html_eng, unsafe_allow_html=True)

                # 라디오 버튼을 클릭하면 해당 doc_id에 대한 HTML을 표시
            show_article_html(selected_doc_id_eng)  

    with tab3:
    # 기부 탭
        # 지역 선택
        region = st.selectbox("기부할 지역 선택", data['city'].unique())

        # 송금금액 입력
        donation_amount = st.number_input("송금할 금액 입력", min_value=0.0)

        # 기부자 정보 입력
        donor_name = st.text_input("기부자 이름 입력")
        donor_email = st.text_input("기부자 이메일 입력")

        # 기부 정보 요약
        st.write("## 기부 정보 요약")
        st.write(f"지역: {region}")
        st.write(f"송금 금액: {donation_amount} 원")
        st.write(f"기부자: {donor_name}")
        st.write(f"이메일: {donor_email}")

        # 기부 버튼
        if st.button("기부하기"):
            # 여기에 기부 처리 로직을 추가하세요
            st.success("기부가 완료되었습니다!")



Overwriting testmapkoreng.py
